In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [2]:
dataset_train = pd.read_csv("../input/trainset.csv")

In [3]:
dataset_train

In [4]:
trainset = dataset_train.iloc[:,1:2].values

In [5]:
trainset

In [6]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0,1))
training_scaled = sc.fit_transform(trainset)

In [7]:
training_scaled

In [8]:
x_train = []
y_train = []

In [9]:
for i in range(60,1259):
    x_train.append(training_scaled[i-60:i, 0])
    y_train.append(training_scaled[i,0])
x_train,y_train = np.array(x_train),np.array(y_train)

In [10]:
x_train.shape

In [11]:
x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))

In [12]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [13]:
regressor = Sequential()
regressor.add(LSTM(units = 50,return_sequences = True,input_shape = (x_train.shape[1],1)))

In [14]:
regressor.add(Dropout(0.2))

In [15]:
regressor.add(LSTM(units = 50,return_sequences = True))
regressor.add(Dropout(0.2))

In [16]:
regressor.add(LSTM(units = 50,return_sequences = True))
regressor.add(Dropout(0.2))

In [17]:
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))

In [18]:
regressor.add(Dense(units = 1))

In [19]:
regressor.compile(optimizer = 'adam',loss = 'mean_squared_error')

In [20]:
regressor.fit(x_train,y_train,epochs = 100, batch_size = 32)

In [21]:
dataset_test =pd.read_csv("../input/testset.csv")

In [22]:
real_stock_price = dataset_test.iloc[:,1:2].values

In [23]:
dataset_total = pd.concat((dataset_train['Open'],dataset_test['Open']),axis = 0)
dataset_total

In [24]:
inputs = dataset_total[len(dataset_total) - len(dataset_test)-60:].values
inputs

In [25]:
inputs = inputs.reshape(-1,1)

In [26]:
inputs

In [27]:
inputs = sc.transform(inputs)
inputs.shape

In [28]:
x_test = []
for i in range(60,185):
    x_test.append(inputs[i-60:i,0])

In [29]:
x_test = np.array(x_test)
x_test.shape

In [30]:
x_test = np.reshape(x_test, (x_test.shape[0],x_test.shape[1],1))
x_test.shape

In [31]:
predicted_price = regressor.predict(x_test)

In [32]:
predicted_price = sc.inverse_transform(predicted_price)
predicted_price

In [33]:
plt.plot(real_stock_price,color = 'red', label = 'Real Price')
plt.plot(predicted_price, color = 'blue', label = 'Predicted Price')
plt.title('Google Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Google Stock Price')
plt.legend()
plt.show()

In [54]:
import numpy as np
pd.DataFrame(data = {"Real":np.squeeze(real_stock_price), "Predicted":np.squeeze(predicted_price)}, index=[i for i in range(len(real_stock_price))])

In [53]:
predicted_price

In [ ]:
.